In [30]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv("Cricket_score_predictor.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91964 entries, 0 to 91963
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battingTeam    91964 non-null  object 
 1   bowlingTeam    91964 non-null  object 
 2   current_score  91964 non-null  int64  
 3   balls          91964 non-null  float64
 4   CRR            91964 non-null  float64
 5   wickets        91964 non-null  int64  
 6   Total_runs     91964 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 4.9+ MB


In [4]:
df.head()

,battingTeam,bowlingTeam,current_score,balls,CRR,wickets,Total_runs
0,Sri Lanka,Pakistan,110,18.3,5.945946,7,132
1,Zimbabwe,Pakistan,91,14.1,6.423529,6,134
2,Sri Lanka,West Indies,115,12.6,8.846154,1,195
3,England,West Indies,144,18.4,7.714286,5,161
4,South Africa,Australia,73,11.6,6.083333,4,146


In [5]:
X = df.drop(columns = ['Total_runs'])

In [6]:
X

,battingTeam,bowlingTeam,current_score,balls,CRR,wickets
0,Sri Lanka,Pakistan,110,18.3,5.945946,7
1,Zimbabwe,Pakistan,91,14.1,6.423529,6
2,Sri Lanka,West Indies,115,12.6,8.846154,1
3,England,West Indies,144,18.4,7.714286,5
4,South Africa,Australia,73,11.6,6.083333,4
...,...,...,...,...,...,...
91959,Australia,South Africa,40,5.1,7.741935,0
91960,Australia,South Africa,3,0.3,6.000000,0
91961,Pakistan,Sri Lanka,37,6.6,5.285714,3
91962,South Africa,Australia,80,13.1,6.075949,4


In [7]:
y = df[['Total_runs']]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 7)

In [10]:
X_test.shape

(18393, 6)

In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,RobustScaler,StandardScaler,PowerTransformer
from sklearn.metrics import r2_score,mean_absolute_error,accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [53]:
step1 = ColumnTransformer(transformers=[
    ('col_tnf',OneHotEncoder(sparse_output=False,drop='first'),[0,1])
],remainder='passthrough')


step2 = XGBRegressor(n_estimators = 6000,
                    learning_rate=0.05 , random_state = 7) 


pipe = Pipeline([
    ('step1',step1),
    ('step2',step2)
])

pipe.fit(X_train,y_train)

y_pred = pipe.predict(X_test)

print('R2 score',r2_score(y_test,y_pred))
print('MAE',mean_absolute_error(y_test,y_pred))

R2 score 0.7963262796401978
MAE 9.498563766479492


In [58]:
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error

# Convert y_test to a 1D array to avoid alignment errors
y_test_arr = np.array(y_test).flatten()
y_pred_arr = np.array(y_pred).flatten()

# 1. Accuracy Percentage (1 - MAPE)
# This gives a value like 94.5%
mape = mean_absolute_percentage_error(y_test_arr, y_pred_arr)
accuracy_pct = (1 - mape) * 100

# 2. Within Range Accuracy
# This calculates what % of predictions are within 10 runs of the actual score
within_10_runs = (np.abs(y_test_arr - y_pred_arr) <= 10).mean() * 100

print(f"R2 Score: {r2_score(y_test, y_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f} runs")
print(f"Accuracy: {accuracy_pct:.2f}%")
print(f"Predictions within 10-run margin: {within_10_runs:.2f}%")

R2 Score: 0.7963
MAE: 9.50 runs
Accuracy: 92.86%
Predictions within 10-run margin: 69.35%


In [56]:
import pickle

pickle.dump(df,open('df.pkl','wb'))
pickle.dump(pipe,open('pipe.pkl','wb'))